## Building Predictive Models

In [1]:
import pandas as pd
import os
import numpy as np

### Import data

In [2]:
#set the path for the processed data
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [3]:
#read the data with all default parameters
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null int64
Deck_B                891 non-null int64
Deck_C                891 non-null int64
Deck_D                891 non-null int64
Deck_E                891 non-null int64
Deck_F                891 non-null int64
Deck_G                891 non-null int64
Deck_Z                891 non-null int64
Pclass_1              891 non-null int64
Pclass_2              891 non-null int64
Pclass_3              891 non-null int64
Title_Lady            891 non-null int64
Title_Master          891 non-null int64
Title_Miss            891 non-null int64
Title_Mr              891 non-null int64
Title_Mrs             891 non-

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null int64
Deck_B                418 non-null int64
Deck_C                418 non-null int64
Deck_D                418 non-null int64
Deck_E                418 non-null int64
Deck_F                418 non-null int64
Deck_G                418 non-null int64
Deck_Z                418 non-null int64
Pclass_1              418 non-null int64
Pclass_2              418 non-null int64
Pclass_3              418 non-null int64
Title_Lady            418 non-null int64
Title_Master          418 non-null int64
Title_Miss            418 non-null int64
Title_Mr              418 non-null int64
Title_Mrs             418 non-null int64
Title_Officer         418 n

### Data Preparation

In [6]:
X = train_df.loc[:,'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel()

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [7]:
print X.shape, y.shape

(891, 32) (891,)


In [9]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(712, 32) (712,)
(179, 32) (179,)


In [10]:
#it is advisable to check the number of positive class values in the initial data model
#average survival rate in train and test
print 'mean survival in train :  {0:.3f}'.format(np.mean(y_train))
print 'mean survival in test :  {0:.3f}'.format(np.mean(y_test))

mean survival in train :  0.383
mean survival in test :  0.385


### Check Scikit-learn version

In [11]:
import sklearn
sklearn.__version__

'0.20.3'

In [12]:
#!conda update -y scikit-learn

### Baseline Model

In [13]:
#import function
from sklearn.dummy import DummyClassifier

In [14]:
#create model
model_dummy = DummyClassifier(strategy = 'most_frequent', random_state=0)

In [15]:
#train model
model_dummy.fit(X_train,y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [16]:
print 'score for baseline model : {0:.2f}'.format(model_dummy.score(X_test,y_test))

score for baseline model : 0.61


In [17]:
#performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [18]:
#accuracy score
print 'accuracy score for baseline model: {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test)))

accuracy score for baseline model: 0.61


In [19]:
#confusion matrix
print 'confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test)))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [20]:
#precision and recall scores
print 'precision score for baseline model: {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test)))
print 'recall score for baseline model: {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test)))

precision score for baseline model: 0.00
recall score for baseline model: 0.00


/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### First Kaggle Submission

In [21]:
#converting to the matrix
test_X = test_df.as_matrix().astype('float')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [22]:
#get predictions
predictions = model_dummy.predict(test_X)

In [23]:
df_submissions = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [24]:
df_submissions.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [25]:
submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
submissions_file_path = os.path.join(submissions_data_path, '01_dummy.csv')

In [27]:
df_submissions.to_csv(submissions_file_path, index=False)

In [28]:
def get_submission_file(model, filename):
    #converting to the matrix
    test_X = test_df.as_matrix().astype('float')
    #make predictions
    predictions = model_dummy.predict(test_X)
    #submissions dataframe
    df_submissions = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    #submissions file
    submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submissions_file_path = os.path.join(submissions_data_path, filename)
    #write to the file
    df_submissions.to_csv(submissions_file_path, index=False)
    

In [29]:
#get submissions file
get_submission_file(model_dummy,'01_dummy.csv')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


### Logistic Regression Model

In [30]:
#import function
from sklearn.linear_model import LogisticRegression

In [31]:
#create model
model_lr_1 = LogisticRegression(random_state = 0)

In [32]:
#train model
model_lr_1.fit(X_train,y_train)

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [33]:
#evaluation model
print 'score for logistic regression - version 1: {0:.2f}'.format(model_lr_1.score(X_test,y_test))

score for logistic regression - version 1: 0.83


In [36]:
# performance metrics
#accuracy score
print 'accuracy score for logistic regression - version 1: {0:.2f}'.format(accuracy_score(y_test, model_lr_1.predict(X_test)))
#confusion matrix
print 'confusion matrix for logistic regression - version 1: \n{0}'.format(confusion_matrix(y_test, model_lr_1.predict(X_test)))
#precision score
print 'precision score for logistic regression - version 1: {0:.2f}'.format(precision_score(y_test, model_lr_1.predict(X_test)))
#recall score
print 'recall score for logistic regression - version 1: {0:.2f}'.format(recall_score(y_test, model_lr_1.predict(X_test)))

accuracy score for logistic regression - version 1: 0.83
confusion matrix for logistic regression - version 1: 
[[95 15]
 [15 54]]
precision score for logistic regression - version 1: 0.78
recall score for logistic regression - version 1: 0.78


In [37]:
#model coefficients (Model Weights or Model Parameters)
model_lr_1.coef_

array([[-0.02842268,  0.00455451, -0.50009089,  0.6178132 , -0.81392331,
         0.12845079, -0.17281789, -0.39317834,  0.52159979,  1.09941224,
         0.40341217, -0.18345052, -0.30036043,  0.96533486,  0.48256744,
        -0.34483448,  0.28089598,  1.21761328,  0.56363966, -1.44586305,
         1.07245548, -0.11273708, -0.47293646,  0.16255648,  0.24716933,
         0.28009428,  0.41324773,  0.49183528,  0.46198829,  0.14924424,
         0.37283516,  0.73023265]])

### Second Kaggle Parameters

In [39]:
#get submission file
get_submission_file(model_lr_1,'02_lr.csv')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
test_X = test_df.as_matrix().astype('float')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [41]:
predictions = model_lr_1.predict(test_X)

In [42]:
df_submissions = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [43]:
submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
submissions_file_path = os.path.join(submissions_data_path, '02_lr.csv')

In [44]:
df_submissions.to_csv(submissions_file_path, index=False)

## Hyperparameter Optimization

In [46]:
#base model
model_lr = LogisticRegression(random_state = 0)

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
parameters = {'C': [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [49]:
clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [50]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [51]:
print 'best score : {0:.2f}'.format(clf.best_score_)

best score : 0.83


In [53]:
#evaluate the model
print 'score for logistic regression - version 2 : {0:.2f}'.format(clf.score(X_test, y_test))

score for logistic regression - version 2 : 0.83


### Making third submission

In [58]:
#get submission file
get_submission_file(clf,'03_lr.csv')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
test_X = test_df.as_matrix().astype('float')

/Users/snigdhakakkar/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [60]:
predictions = clf.predict(test_X)

In [61]:
df_submissions = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [62]:
submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
submissions_file_path = os.path.join(submissions_data_path, '03_lr.csv')

In [63]:
df_submissions.to_csv(submissions_file_path, index=False)

### Feature Normalization and Standardization

In [64]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature Normalization

In [65]:
#feature normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [66]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [67]:
#normalize the test data as well
X_test_scaled = scaler.transform(X_test)

#### Feature Standardization

In [68]:
#feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Create model after standardization

In [69]:
#base model
model_lr = LogisticRegression()
parameters = {'C': [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [70]:
clf.best_score_

0.8132022471910112

In [71]:
#evaluate the model
print 'score for logistic regression - version 2 : {0:.2f}'.format(clf.score(X_test_scaled, y_test))

score for logistic regression - version 2 : 0.84


## Model Persistence

In [72]:
#import pickle library
import pickle

In [73]:
#create the file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')

In [74]:
#open the files to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [75]:
#persist the model and the scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [76]:
#close the files
model_file_pickle.close()
scaler_file_pickle.close()

#### Load the persisted file

In [77]:
#open files in read mode
model_file_pickle = open(model_file_path, 'r')
scaler_file_pickle = open(scaler_file_path, 'r')
#load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
#close the files
model_file_pickle.close()
scaler_file_pickle.close()

In [78]:
clf_loaded

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [79]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [81]:
#transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
#calculate the score using the loaded scaler object
print 'score for logistic regression - version 2 : {0:.2f}'.format(clf_loaded.score(X_test_scaled, y_test))

score for logistic regression - version 2 : 0.84
